In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
dataPath='/content/drive/MyDrive/KITA_2024/M10_LLama/dataset/'
# !mkdir -p {dataPath}

### Library 설치

In [2]:
!pip cache purge
!pip uninstall fsspec gcsfs -y
!pip install fsspec==2024.6.1 gcsfs==2024.6.1

Files removed: 0
Found existing installation: fsspec 2024.6.1
Uninstalling fsspec-2024.6.1:
  Successfully uninstalled fsspec-2024.6.1
Found existing installation: gcsfs 2024.6.1
Uninstalling gcsfs-2024.6.1:
  Successfully uninstalled gcsfs-2024.6.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 3.3 MB/s eta 0:00:00


In [4]:
!pip install datasets==2.16.1 jsonlines==4.0.0 -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.6.1 requires fsspec==2024.6.1, but you have fsspec 2023.10.0 which is incompatible.


#### huggingface 접근을 위한 로그인
- 데이터셋을 가져다 놓고 쓰기 위해..

In [5]:
import huggingface_hub
huggingface_hub.login()

#### 설치한 라이브러리 선언

In [6]:
import pandas as pd
import json
import jsonlines
from datasets import Dataset

#### 1. 학습 데이터 불러오기 및 변환

In [7]:
datasetName="train_1.csv"
jsonFileName="train_1.jsonl"

In [14]:
def csv_to_jsonl(csv_file_path, jsonl_file_path):
    # CSV 파일을 읽어옵니다.
    df = pd.read_csv(csv_file_path, encoding='utf-8')

    with open(jsonl_file_path, 'w', encoding='utf-8') as jsonl_file:
      for index, row in df.iterrows():
        data = {
            "input": row['질문'], "response": row['답변']}
        json.dump(data, jsonl_file, ensure_ascii=False)
        jsonl_file.write('\n')

csv_file_path = dataPath + datasetName
json_file_path = dataPath + jsonFileName
csv_to_jsonl(csv_file_path, json_file_path)

#### 2. 파인튜닝용 포맷으로 변환

In [16]:
indataset = []
with jsonlines.open(json_file_path) as f:
  for line in f.iter():
    indataset.append(f'<s>[INST] {line["input"]} [/INST] {line["response"]} </s>')


print("데이터셋 확인")
print(indataset[:5])

# 데이터셋 생성 및 저장
indataset = Dataset.from_dict({"text": indataset})
indataset.save_to_disk(dataPath)

# 데이터넷 info 확인
print("데이터셋 info 확인")
print(indataset)

데이터셋 확인
['<s>[INST] 면진장치가 뭐야? [/INST] 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다. </s>', '<s>[INST] 면진장치에 사용되는 주요 기술은 무엇인가요? [/INST] 면진장치란 지반에서 오는 진동 에너지를 흡수하여 건물에 주는 진동을 줄여주는 진동 격리장치입니다. </s>', '<s>[INST] 내진설계의 종류 좀 알려줘 [/INST] 내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다. </s>', '<s>[INST] 내진설계에는 어떤 종류가 있는지 자세히 알려주실 수 있나요? [/INST] 내진 설계의 종류로 내진구조, 제진구조, 면진구조가 있습니다. </s>', '<s>[INST] 철골구조의 장점이 뭐야? [/INST] 철골구조는 건물의 외벽에는 그다지 하중이 걸리지 않기 때문에 고층 건물의 건축이 가능한 것이 장점입니다. </s>']


Saving the dataset (0/1 shards):   0%|          | 0/1288 [00:00<?, ? examples/s]

데이터셋 info 확인
Dataset({
    features: ['text'],
    num_rows: 1288
})


#### 3. huggingfasce에 데이터 업로드

In [17]:
indataset.push_to_hub("hwooks/llama2dataset1")      ### MyID: hwooks 사용

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/hwooks/llama2dataset1/commit/a8d6c97e464663b4a5e9f3b173d61a41448fa469', commit_message='Upload dataset', commit_description='', oid='a8d6c97e464663b4a5e9f3b173d61a41448fa469', pr_url=None, pr_revision=None, pr_num=None)